In [1]:
import  pandas as pd
import  numpy as np
train=pd.read_csv('smallTrain.csv')

In [2]:
train.head(10)

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,...,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,year,month,day
0,2,3,66,348,48862,2234.2641,12,0,1,9,...,1,0,3,2,50,628,1,2014,8,11
1,2,3,66,348,48862,2234.2641,12,0,1,9,...,1,1,1,2,50,628,1,2014,8,11
2,2,3,66,348,48862,2234.2641,12,0,0,9,...,1,0,1,2,50,628,1,2014,8,11
3,2,3,66,442,35390,913.1932,93,0,0,3,...,1,0,1,2,50,1457,80,2014,8,9
4,2,3,66,442,35390,913.6259,93,0,0,3,...,1,0,1,2,50,1457,21,2014,8,9
5,2,3,66,442,35390,911.5142,93,0,0,3,...,1,0,1,2,50,1457,92,2014,8,9
6,2,3,66,189,10067,-13.0000,501,0,0,2,...,1,0,2,2,50,675,41,2014,7,16
7,2,3,66,189,10067,-13.0000,501,0,1,2,...,1,0,1,2,50,675,41,2014,7,16
8,2,3,66,189,10067,-13.0000,501,0,0,2,...,1,0,1,2,50,675,69,2014,7,16
9,2,3,66,189,10067,-13.0000,501,0,0,2,...,1,0,1,2,50,675,70,2014,7,16


In [3]:
y=train['hotel_cluster']
train=train.drop('hotel_cluster',axis=1)
train=train.drop('orig_destination_distance',axis=1)

In [4]:
train.shape

(100000, 24)

In [5]:
#copy pasted from google 
from math import factorial
def combination(n,k):
    numerator=factorial(n)
    denominator=(factorial(k)*factorial(n-k))
    answer=numerator/denominator
    return answer

In [6]:
combination(4,3)

4.0

In [7]:
N=train.shape[1]

In [8]:
N

24

In [9]:
import itertools

In [10]:
#GENERATING ALL COMBINATIONS  - willl  bring  back     3D indexes of column  names
comb=itertools.combinations(range(N), 3)
idx=[ elem   for  elem in  comb]

print(len(idx))


2024


In [11]:
from random import randrange
random_index = randrange(0,len(idx))
random_joint=idx[random_index]

In [12]:
random_joint=np.array(random_joint)
random_joint

array([ 0,  9, 10])

In [13]:
names=train.columns
names

Index(['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile',
       'is_package', 'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_id',
       'srch_destination_type_id', 'is_booking', 'cnt', 'hotel_continent',
       'hotel_country', 'hotel_market', 'year', 'month', 'day'],
      dtype='object')

In [14]:
names_subset=names[list(random_joint)]

In [15]:
tr=train[['is_mobile', 'srch_rm_cnt', 'hotel_continent']]
tr.head(3)

,is_mobile,srch_rm_cnt,hotel_continent
0,0,1,2
1,0,1,2
2,0,1,2


In [84]:
from collections import Counter
class featuresfrom3d(object):
    def __init__(self,data):
        """ init   class  with  3D  DATA FRAME  and  other  data  members  needed"""
        self.data=data
        # unique   categories for  the  3 features
        self.u1=None
        self.u2=None
        self.u3=None
        #names  for   easy call   of the columns
        self.name1=self.data.columns[0]
        self.name2=self.data.columns[1]
        self.name3=self.data.columns[2]
        #mapping     features
        self.u1_mapping=None
        self.u2_mapping=None
        self.u3_mapping=None
        #dimmensions  of    the   3D joint probability distro is    basically  the  cardinality of the   categories
        self.dimmension_x=None
        self.dimmension_y=None
        self.dimmension_z=None
        #set prob_space
        self.prob_space=None
        #the joint  distribution   of the   3  randomly choosen features
        self.joint_prob_table=None
    def  unique_cat(self):
        """ get unique   categories   for  the    3 columns as list"""
        self.u1=list(np.unique(self.data[self.name1]))
        
        self.u2=list(np.unique(self.data[self.name2]))
        
        self.u3=list(np.unique(self.data[self.name3]))
        #after   computing the unique categories   will  add also    dimmensions of lego ,, cube''
        self.dimmension_x=len(self.u1)
        self.dimmension_y=len(self.u2)
        self.dimmension_z=len(self.u3)
    def mapping(self):
        """maps  with indexes  the        data  members    with   3 unique   values in order  to put   them  
         in  a     3 dimmensional    lego """
        
        self.u1_mapping={label:index
           for  index,label in  enumerate(self.u1)}
        
        self.u2_mapping={label:index
           for  index,label in  enumerate(self.u2)}
        
        self.u3_mapping={label:index
           for  index,label in  enumerate(self.u3)}
    def probspace(self):
        """ Generates  probability  space
        Here   is the    most  time extensive part , but   with  COunter   it is  solving in less  then  a second 
        """
        data=np.array(self.data)
        tuples = [tuple(row) for row in data]
        #get  frequency  for  each row   , Counter  fast  enough   for  our   task 
        frequency_of_rows=Counter(tuples)
        
        
        probs={key:value/self.data.shape[0] for key,value in frequency_of_rows.items()}
        self.prob_space=probs
        #return  probs
    
    def  create_joint_3dtable(self):
        #creates  the   joint   3D  table 
        
        self.joint_prob_table=np.zeros((self.dimmension_x,self.dimmension_y,self.dimmension_y))
        
        for x,y, z in self.prob_space:
            self.joint_prob_table[self.u1_mapping[x],self.u2_mapping[y],self.u3_mapping[y]]=self.prob_space[(x,y,z)]
        
        
    def  conditioning1(self):
        a=1
        """must  do conditioning   and  return coresponding  feature both for train  and  test"""
        #to do
    def  conditioning2(self):
        a=1
        #to do
    def  conditioning3(self):
        a=1
        #to do
    def  conditioning4(self):
        a=1
        #to do
class  GenerateFeatures(featuresfrom3d):
    def __init__(self,N_COLUMNS,data):
        """  INHERITS   EVERYTHING FROM   FEATURESFROM3D CLASS"""
        self.N_COLUMNS=N_COLUMNS
        featuresfrom3d.__init__(self,data)
        

In [85]:
s=featuresfrom3d(tr)

In [86]:
s.unique_cat()

In [87]:
s.mapping()

In [110]:
map1=s.u1_mapping
map2=s.u2_mapping
map3=s.u3_mapping
prob_space=s.prob_space

In [109]:
joint_prob=s.joint_prob_table
joint_prob.shape

(2, 9, 9)

In [111]:
for  x,y,z in prob_space:
    joint_prob[map1[x],
               map2[y],
               map3[z]]=prob_space[(x,y,z)]

In [112]:
joint_prob

array([[[  0.00000000e+00,   3.00000000e-05,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  9.52000000e-03,   4.18160000e-01,   9.45300000e-02,
           9.21300000e-02,   9.20000000e-03,   1.60580000e-01,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.80000000e-04,   3.16700000e-02,   7.79000000e-03,
           5.87000000e-03,   1.26000000e-03,   1.75500000e-02,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  1.80000000e-04,   4.53000000e-03,   2.46000000e-03,
           5.60000000e-04,   1.00000000e-05,   1.28000000e-03,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.00000000e-04,   4.30000000e-04,
           9.00000000e-05,   3.00000000e-05,   2.70000000e-04,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  0.00000000e+00,   9.10000000e-04,   1.0